In [55]:
#Put all imports here
import numpy as np
import matplotlib.pyplot as plt
#from copy import deepcopy
#import pickle
import os
import sys
#import cv2
import torch
import csv

In [56]:
connect_to_drive = False

In [57]:
#every path should start from the project folder:
project_folder = "../"
if connect_to_drive:
    project_folder = "/content/gdrive/Shareddrives/<SharedDriveName>" #Name of SharedDrive folder
    #project_folder = "/content/gdrive/MyDrive/<MyDriveName>" #Name of MyDrive folder

#Config folder should contain hyperparameters configurations
cfg_folder = os.path.join(project_folder,"cfg")

#Data folder should contain raw and preprocessed data
data_folder = os.path.join(project_folder,"data")
raw_data_folder = os.path.join(data_folder,"raw")
processed_data_folder = os.path.join(data_folder,"processed")

#Source folder should contain all the (essential) source code
source_folder = os.path.join(project_folder,"src")

#The out folder should contain all outputs: models, results, plots, etc.
out_folder = os.path.join(project_folder,"out")
img_folder = os.path.join(out_folder,"img")

In [58]:
#To import from src:

#attach the source folder to the start of sys.path
sys.path.insert(0, project_folder)

#import from src directory
from src.module import *

import easy_exp, easy_rec, easy_torch #easy_data

In [59]:
cfg = easy_exp.cfg.load_configuration("config_rec")

In [60]:
exp_found, experiment_id = easy_exp.exp.get_set_experiment_id(cfg)

In [61]:
data_params = deepcopy(cfg["data_params"])
data_params["data_folder"] = raw_data_folder

In [62]:
data, maps = easy_rec.data_generation_utils.preprocess_dataset(**data_params)

Ratings data already exists. Skip pre-processing
Filtering by minimum number of users per item: 5
Filtering by minimum number of items per user: 5
Densifying index
Splitting: leave_n_out


In [63]:
datasets = easy_rec.rec_torch.prepare_rec_datasets(data,**cfg["data_params"]["dataset_params"])

In [64]:
num_users = np.max(list(maps["uid"].values()))
num_items = np.max(list(maps["sid"].values()))

utility_matrix = create_utility_matrix_from_dataset(datasets['train'], num_users, num_items)

In [65]:
print(cfg['data_params']['name'])
print(num_users)
print(num_items)

foursquare-tky
2293
15177


In [66]:
from sklearn import cluster

model = cluster.HDBSCAN(min_samples=1,n_jobs=-1)
y_hat = model.fit_predict(utility_matrix)

In [67]:
print(len(y_hat))

n_clusters = len(np.unique(y_hat)) 
print(f"#users clusters found: {n_clusters}")

2293
#users clusters found: 1


In [68]:
from sklearn import cluster

model_i = cluster.HDBSCAN(min_samples=1,n_jobs=-1)
y_hat_i = model_i.fit_predict(utility_matrix.T)

In [69]:
print(len(y_hat_i))

n_clusters_i = len(np.unique(y_hat_i))
print(f"#items clusters found: {n_clusters_i}")

15177
#items clusters found: 283
